In [33]:
import nest_asyncio
nest_asyncio.apply()
from pathlib import Path
from datetime import datetime
import re

import sys
sys.path.append("../")

from desci_sense.shared_functions.init import init_multi_chain_parser_config
from desci_sense.evaluation.Evaluation_benchmark import TwitterEval
from desci_sense.shared_functions.parsers.multi_chain_parser import MultiChainParser
from desci_sense.evaluation.utils import get_dataset, obj_to_json, obj_str_to_dict
from desci_sense.shared_functions.dataloaders import (
    scrape_post,
    convert_text_to_ref_post,
)

In [2]:
config = init_multi_chain_parser_config(llm_type='mistralai/mixtral-8x7b',
                                        post_process_type="combined")

In [3]:
multi_chain_parser = MultiChainParser(config)

2024-05-28 13:21:04.915 | INFO     | desci_sense.shared_functions.parsers.multi_chain_parser:__init__:66 - Initializing MultiChainParser. PostProcessType=combined
2024-05-28 13:21:04.921 | INFO     | desci_sense.shared_functions.parsers.multi_chain_parser:__init__:73 - Initializing post parsers...
2024-05-28 13:21:04.922 | INFO     | desci_sense.shared_functions.parsers.post_parser_chain:__init__:26 - Initializing parser chain 'multi_refs_tagger' 
2024-05-28 13:21:04.969 | INFO     | desci_sense.shared_functions.parsers.post_parser_chain:__init__:26 - Initializing parser chain 'topics' 
2024-05-28 13:21:04.977 | INFO     | desci_sense.shared_functions.parsers.post_parser_chain:__init__:26 - Initializing parser chain 'keywords' 
2024-05-28 13:21:04.986 | INFO     | desci_sense.shared_functions.parsers.post_parser_chain:__init__:26 - Initializing parser chain 'hashtags' 


## Loading wandb dataset to test

In [32]:
import wandb
import pandas as pd

wandb.login()

api = wandb.Api()

#TODO move from testing
run = wandb.init(project="testing", job_type="evaluation")

# get artifact path

dataset_artifact_id = (
        'common-sense-makers/filter_evaluation/labeled_tweets_no_threads:v1'
    )

# set artifact as input artifact
dataset_artifact = run.use_artifact(dataset_artifact_id)

# initialize table path
# add the option to call table_path =  arguments.get('--dataset')

# download path to table
a_path = dataset_artifact.download()
print("The path is",a_path)

# get dataset file name

table_path = Path(f"{a_path}/labeled_data_table_no_threads.table.json")


# return the pd df from the table
#remember to remove the head TODO
df = get_dataset(table_path).head(150)

table_path = Path(f"{a_path}/handles_chart.table.json")

df_handles = get_dataset(table_path)



wandb:   2 of 2 files downloaded.  


The path is /Users/shaharorielkagan/sensemakers/nlp/notebooks/artifacts/labeled_tweets_no_threads:v1


In [33]:
import concurrent.futures
from tqdm import tqdm

def process_text(text):
    return convert_text_to_ref_post(text)

# Assuming df['Text'] is your DataFrame column
with concurrent.futures.ThreadPoolExecutor() as executor:
    inputs = list(tqdm(executor.map(process_text, df['Text']), total=len(df['Text'])))


100%|██████████| 150/150 [00:18<00:00,  8.11it/s]


In [34]:
post = scrape_post('https://x.com/rtk254/status/1741841607421263966')

In [35]:
results = multi_chain_parser.process_ref_post(post=post,active_list=["hashtags"])

2024-06-04 12:19:46.811 | WARNING  | desci_sense.shared_functions.web_extractors.citoid:before_retry:84 - Retry attempt 1
2024-06-04 12:19:46.815 | DEBUG    | desci_sense.shared_functions.web_extractors.citoid:fetch_citation_async_retry:111 - target_url=https://paragraph.xyz/@sense-nets/2-project-plan
2024-06-04 12:19:46.827 | WARNING  | desci_sense.shared_functions.web_extractors.citoid:before_retry:84 - Retry attempt 1
2024-06-04 12:19:46.828 | DEBUG    | desci_sense.shared_functions.web_extractors.citoid:fetch_citation_async_retry:104 - skipping citoid for https://twitter.com/i/status/1732044124508008571
2024-06-04 12:19:46.831 | WARNING  | desci_sense.shared_functions.web_extractors.citoid:before_retry:84 - Retry attempt 1
2024-06-04 12:19:46.834 | DEBUG    | desci_sense.shared_functions.web_extractors.citoid:fetch_citation_async_retry:111 - target_url=https://paragraph.xyz/@sense-nets/sense-nets-intro
2024-06-04 12:19:47.869 | DEBUG    | desci_sense.shared_functions.parsers.multi_

In [36]:
Eval = TwitterEval(config=config)
urls = results.reference_urls
quotes = Eval.nested_quotes_citoid
print(quotes)

AttributeError: 'TwitterEval' object has no attribute 'nested_quotes_citoid'

In [32]:
results

CombinedParserOutput(research_keyword='not-detected', filter_classification=<SciFilterClassfication.NOT_RESEARCH: 'not_research'>, item_types=['webpage', 'forumPost', 'webpage'], reference_urls=['https://paragraph.xyz/@sense-nets/2-project-plan', 'https://twitter.com/i/status/1732044124508008571', 'https://paragraph.xyz/@sense-nets/sense-nets-intro'], reference_tagger=None, multi_reference_tagger=None, keywords=[], topics=[], hashtags=[], metadata_list=[RefMetadata(citoid_url='https://paragraph.xyz/@sense-nets/2-project-plan', url='https://paragraph.xyz/@sense-nets/2-project-plan', item_type='webpage', title='Part 2: Sensemaking Networks Project Plan', summary='Integrating fragmented science social media and reducing the barriers to semantic publishing', image='', debug={'error': None}), RefMetadata(citoid_url='https://twitter.com/i/status/1732044124508008571', url='https://twitter.com/i/status/1732044124508008571', item_type='forumPost', title='Twitter post', summary='', image='', deb

In [24]:
def check_quotes1(urls:list):
        quotes = []
        pattern = re.compile(r'^https://twitter\.com/.+/[0-9]+$')
        for url in urls:
            if pattern.match(url):
                quotes.append(url)
        return quotes

In [25]:
quotes = check_quotes1(urls)

In [26]:
print(quotes)

['https://twitter.com/i/status/1732044124508008571']


In [36]:
# batch process
results = multi_chain_parser.batch_process_ref_posts(inputs=inputs[:20],active_list=["keywords", "topics"],batch_size=10)

2024-05-21 09:24:12.720 | WARNING  | desci_sense.shared_functions.web_extractors.citoid:before_retry:84 - Retry attempt 1
2024-05-21 09:24:12.724 | DEBUG    | desci_sense.shared_functions.web_extractors.citoid:fetch_citation_async_retry:111 - target_url=https://en.wikipedia.org/wiki/Epic_of_evolution
2024-05-21 09:24:12.731 | WARNING  | desci_sense.shared_functions.web_extractors.citoid:before_retry:84 - Retry attempt 1
2024-05-21 09:24:12.731 | DEBUG    | desci_sense.shared_functions.web_extractors.citoid:fetch_citation_async_retry:111 - target_url=https://wiki.p2pfoundation.net/Category:Peerproduction#With_the_advent_of_the_P2P_Mode_of_Production%2C_the_community_and_its_common_is_now_the_appropriate_scale
2024-05-21 09:24:12.732 | WARNING  | desci_sense.shared_functions.web_extractors.citoid:before_retry:84 - Retry attempt 1
2024-05-21 09:24:12.733 | DEBUG    | desci_sense.shared_functions.web_extractors.citoid:fetch_citation_async_retry:111 - target_url=https://integralpermaculture

  0%|          | 0/40 [00:00<?, ?it/s]

2024-05-21 09:24:30.013 | DEBUG    | desci_sense.shared_functions.parsers.multi_chain_parser:batch_process_ref_posts:248 - Invoking parallel chain...
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/typing.py:373: RuntimeWarning: coroutine 'Runnable.abatch.<locals>.ainvoke' was never awaited
  ev_args = tuple(_eval_type(a, globalns, localns, recursive_guard) for a in t.__args__)


KeyboardInterrupt: 

In [30]:
print(results[1].debug['keywords']['reasoning'])

[Reasoning Steps]

**

* Analyzed the post's content to identify key concepts and issues.
* Considered both general and specific keywords relevant to the topics discussed.
* Included a special keyword to indicate if the post is related to academic research.


**

[Candidate Keywords]

**

* #sustainability
* #naturalcapital
* #resourcemanagement
* #socialcapital
* #regenerativeculture
* #conservation
* #consumption


**


In [ ]:
for p in results:
    try:
        p.debug['topics']['full_text']
    except:
        print(p.debug['topics'])

In [24]:
# update predictions to dataframe
df['Predicted Label'] = [x.filter_classification.value for x in results]
df['Reasoning Steps'] = ["Keywords: "+str(x.debug['topics']['reasoning'])+"Topics: "+str(x.debug['keywords']['reasoning']) for x in results]
df['Keywords'] = [x.keywords for x in results]
df['Topics'] = [x.topics for x in results]
df['Ref item types'] = [x.item_types for x in results]
df['academic_keyword'] = [x.research_keyword for x in results]




In [74]:
config = {'batch_size': 5, 'wandb_config': {'entity': 'common-sense-makers', 'project': 'st-demo-sandbox'}, 'parser_configs': {'0': {'name': 'refs_tagger', 'type': '<ParserChainType.REFERENCE_TAGGER:', 'llm_config': {'llm_type': 'random_parser'}, 'temperature': 0.5, 'use_metadata': True, 'is_ref': True}, '1': {'name': 'multi_refs_tagger', 'type': '<ParserChainType.MULTI_REF_TAGGER:', 'llm_config': {'llm_type': 'mistralai/mixtral-8x7b-instruct:nitro'}, 'temperature': 0.6, 'use_metadata': True, 'is_multi_ref': True}, '2': {'name': 'topics', 'type': '<ParserChainType.TOPICS:', 'llm_config': {'llm_type': 'mistralai/mixtral-8x7b-instruct:nitro'}, 'temperature': 0.6, 'use_metadata': True, 'is_topic': True}, '3': {'name': 'keywords', 'type': '<ParserChainType.KEYWORDS:', 'llm_config': {'llm_type': 'mistralai/mixtral-8x7b-instruct:nitro'}, 'temperature': 0.6, 'use_metadata': True, 'max_keywords': 6}, '4': {'name': 'hashtags', 'type': '<ParserChainType.HASHTAGS:', 'llm_config': {'llm_type': 'mistralai/mistral-7b-instruct'}, 'temperature': 0.6, 'use_metadata': False, 'max_hashtags': 20}}, 'post_process_type': 'PostProcessType.COMBINED', 'openrouter_api_config': {'openrouter_api_base': 'https://openrouter.ai/api/v1', 'openrouter_api_key': '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', 'openrouter_referer': 'http://localhost:3000'}, 'metadata_extract_config': {'extraction_method': '<MetadataExtractionType.CITOID:', 'max_summary_length': 500}}

In [25]:
str(None)

'None'

In [26]:
# Log dataframe in wandb
wandb.login()

api = wandb.Api()

#TODO move from testing
run = wandb.init(project="filter_evaluation", job_type="predictions")

# Create the evaluation artifact
current_datetime = datetime.now()
time = current_datetime.strftime("%Y%m%d%H%M%S")


artifact = wandb.Artifact("prediction_evaluation-" + str(time), type="evaluation")

# Create a wandb.Table from the Pandas DataFrame
table = wandb.Table(dataframe=df)

# Add the wandb.Table to the artifact
artifact.add(table, "prediction_evaluation")

# model_info is your model metadata
run.config.update(config)

 # Log the artifact
wandb.log_artifact(artifact, aliases=["latest"])

wandb.run.finish()

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


In [ ]:
bad_posts = []
for i in range(20,len(df["Text"])):
    print(i)
    p = df['Text'][i]
    r = convert_text_to_ref_post(p)
    try:
        result=multi_chain_parser.process_ref_post(r,active_list=["topics",'keywords'])
    except Exception as e:
        print("Post with text: ",p)
        print('and url: ',df['postURL'][i])
        print("has raised an exception: ",e) 
        bad_posts.append((df['postURL'][i],e))
          


    

In [ ]:
print(bad_posts)

[]


Python(28593) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(28594) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(28595) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(28596) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(28597) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(28598) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(28599) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(28600) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(28601) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(28602) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(28603) Malloc

In [40]:
print(config)

{}


In [ ]:
def json_config_construct(config):
    

In [21]:
import json
import inspect

def obj_to_dict(obj):
    if isinstance(obj, dict):
        return {k: obj_to_dict(v) for k, v in obj.items()}
    elif hasattr(obj, "__dict__"):
        return {k: obj_to_dict(v) for k, v in obj.__dict__.items() if not k.startswith('_')}
    elif isinstance(obj, list):
        return [obj_to_dict(i) for i in obj]
    elif isinstance(obj, tuple):
        return tuple(obj_to_dict(i) for i in obj)
    elif isinstance(obj, set):
        return {obj_to_dict(i) for i in obj}
    else:
        return obj

def obj_to_json(obj):
    return json.dumps(obj_to_dict(obj), indent=4)

# Example classes
class Address:
    def __init__(self, street, city):
        self.street = street
        self.city = city

class Person:
    def __init__(self, name, age, address):
        self.name = name
        self.age = age
        self.address = address

# Example usage
address = Address("123 Main St", "Springfield")
person = Person("John Doe", 30, address)

print(obj_to_json(person))


{
    "name": "John Doe",
    "age": 30,
    "address": {
        "street": "123 Main St",
        "city": "Springfield"
    }
}


In [42]:
print(obj_to_dict(config))

{'openrouter_api_config': {'openrouter_api_base': 'https://openrouter.ai/api/v1', 'openrouter_api_key': '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', 'openrouter_referer': 'http://localhost:3000'}, 'parser_configs': [{'name': 'multi_refs_tagger', 'type': {}, 'llm_config': {'llm_type': 'openai/gpt-3.5-turbo-0125', 'temperature': '0.6'}, 'use_metadata': True, 'is_multi_ref': True}, {'name': 'topics', 'type': {}, 'llm_config': {'llm_type': 'openai/gpt-3.5-turbo-0125', 'temperature': '0.6'}, 'use_metadata': True, 'is_topic': True}, {'name': 'keywords', 'type': {}, 'llm_config': {'llm_type': 'openai/gpt-3.5-turbo-0125', 'temperature': '0.6'}, 'use_metadata': True, 'max_keywords': 6}, {'name': 'hashtags', 'type': {}, 'llm_config': {'llm_type': 'mistralai/mistral-7b-instruct', 'temperature': '0.6'}, 'use_metadata': False, 'max_hashtags': 20}], 'metadata_extract_config': {'extraction_method': {}, 'max_summary_length': 500}, 'wandb_config': {'entity': 'common-sens

In [53]:
import re
import ast
import wandb

def parse_string_to_dict(s):
    """Parses a string representation of a dictionary or object."""
    try:
        return ast.literal_eval(s)
    except Exception:
        pass
    
    # Custom parsing logic for object-like strings
    result = {}
    # Split by spaces, but keep quoted strings together
    parts = re.findall(r'(\w+)=\'(.*?)\'|(\w+)=([^\s]+)', s)
    
    for part in parts:
        if part[0]:
            key, value = part[0], part[1]
        else:
            key, value = part[2], part[3]
        
        # Attempt to parse the value further
        try:
            value = ast.literal_eval(value)
        except Exception:
            if isinstance(value, str) and '=' in value:
                value = parse_string_to_dict(value)
        
        result[key] = value
    
    return result if result else s

def obj_to_dict(obj):
    if isinstance(obj, dict):
        return {k: obj_to_dict(v) for k, v in obj.items()}
    elif hasattr(obj, "__dict__"):
        return {k: obj_to_dict(v) for k, v in obj.__dict__.items() if not k.startswith('_')}
    elif isinstance(obj, list):
        return [obj_to_dict(i) for i in obj]
    elif isinstance(obj, tuple):
        return tuple(obj_to_dict(i) for i in obj)
    elif isinstance(obj, set):
        return {obj_to_dict(i) for i in obj}
    elif isinstance(obj, str):
        return parse_string_to_dict(obj)
    else:
        return obj


In [64]:
import wandb
import json


# Ensure you're logged into W&B
wandb.login()

# Set your project and entity names
project_name = 'filter_evaluation'
entity_name = 'common-sense-makers'

# Retrieve all runs in the project
api = wandb.Api()
runs = api.runs(f"{entity_name}/{project_name}")

# Iterate through each run and apply the transformation
for run in runs:
    # Retrieve the current configuration
    config = dict(run.config)
    print('config: ',config)
    print('+++++')
    
    # Apply the transformation
    transformed_config = obj_to_dict(config)
    print('transformed_config: ', transformed_config)
    print('++++++++')
    # Update the run configuration
    try: 
        run.config.clear()
        run.config['llm_config']=transformed_config['parser_configs'][0]['llm_config']
        print('OUT')
    except Exception as e:
        print(e)
    # Optionally, save the updated run (Note: wandb.run object must be used with `wandb.init`)
    # run.update() # This is just a placeholder, not an actual method

# Print a message when done
print("All runs have been updated.")


config:  {}
+++++
transformed_config:  {}
++++++++
'parser_configs'
config:  {}
+++++
transformed_config:  {}
++++++++
'parser_configs'
config:  {}
+++++
transformed_config:  {}
++++++++
'parser_configs'
config:  {}
+++++
transformed_config:  {}
++++++++
'parser_configs'
config:  {'batch_size': 5, 'wandb_config': "entity='common-sense-makers' project='st-demo-sandbox'", 'parser_configs': ["name='multi_refs_tagger' type=<ParserChainType.MULTI_REF_TAGGER: 'multi_reference_tagger'> llm_config=LLMConfig(llm_type='openai/gpt-3.5-turbo-0125', temperature='0.6') use_metadata=True is_multi_ref=True", "name='topics' type=<ParserChainType.TOPICS: 'topics'> llm_config=LLMConfig(llm_type='openai/gpt-3.5-turbo-0125', temperature='0.6') use_metadata=True is_topic=True", "name='keywords' type=<ParserChainType.KEYWORDS: 'keywords'> llm_config=LLMConfig(llm_type='openai/gpt-3.5-turbo-0125', temperature='0.6') use_metadata=True max_keywords=6", "name='hashtags' type=<ParserChainType.HASHTAGS: 'hashtags'

In [59]:
ex = obj_to_dict({'batch_size': 5, 'wandb_config': "entity='common-sense-makers' project='st-demo-sandbox'", 'parser_configs': ["name='refs_tagger' type=<ParserChainType.REFERENCE_TAGGER: 'reference_tagger'> llm_config=LLMConfig(llm_type='google/gemma-7b-it:nitro', temperature='0.6') use_metadata=True is_ref=True", "name='multi_refs_tagger' type=<ParserChainType.MULTI_REF_TAGGER: 'multi_reference_tagger'> llm_config=LLMConfig(llm_type='google/gemma-7b-it:nitro', temperature='0.6') use_metadata=True is_multi_ref=True", "name='topics' type=<ParserChainType.TOPICS: 'topics'> llm_config=LLMConfig(llm_type='google/gemma-7b-it:nitro', temperature='0.6') use_metadata=True is_topic=True", "name='keywords' type=<ParserChainType.KEYWORDS: 'keywords'> llm_config=LLMConfig(llm_type='google/gemma-7b-it:nitro', temperature='0.6') use_metadata=True max_keywords=6", "name='hashtags' type=<ParserChainType.HASHTAGS: 'hashtags'> llm_config=LLMConfig(llm_type='mistralai/mistral-7b-instruct', temperature='0.6') use_metadata=False max_hashtags=20"], 'post_process_type': 'PostProcessType.COMBINED', 'openrouter_api_config': "openrouter_api_base='https://openrouter.ai/api/v1' openrouter_api_key='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' openrouter_referer='http://localhost:3000'", 'metadata_extract_config': "extraction_method=<MetadataExtractionType.CITOID: 'citoid'> max_summary_length=500"})


In [62]:
print(ex['parser_configs'][0]['llm_config'])

{'llm_type': 'google/gemma-7b-it:nitro'}


In [67]:
# random parser
import pandas as pd
import numpy as np

# Assuming you have a DataFrame 'df' with your true labels
true_labels = df['True Label']

#Generate random predictions
np.random.seed(42)  # For reproducibility
random_predictions = np.random.choice(['research', 'not_research'], size=len(true_labels))


# Add these random predictions to the DataFrame
df['Predicted Label'] = random_predictions



In [48]:
# Import bad prediction table

wandb.login()

api = wandb.Api()

#TODO move from testing
run = wandb.init(project="filter_evaluation", job_type="evaluation")
dataset_artifact_id = (
        'common-sense-makers/filter_evaluation/prediction_evaluation-20240521132713:v0'
    )
dataset_artifact = run.use_artifact(dataset_artifact_id)

a_path = dataset_artifact.download()

# get handle file name

table_path = Path(f"{a_path}/prediction_evaluation.table.json")

dataset_run = dataset_artifact.logged_by()

config = dataset_run.config

df = get_dataset(table_path)

wandb.run.finish()

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


wandb:   1 of 1 files downloaded.  


In [81]:
keys = set(df2.columns) - set(df.columns)
for key in keys:
    df[key] = df2[key]

In [82]:
df = df.drop(columns=['random_predictions'])

In [41]:
import re

def check_twitter_urls(urls):
    pattern = re.compile(r'^https://twitter\.com/.+/[0-9]+$')
    for url in urls:
        if pattern.match(url):
            return 1
    return 0

SyntaxError: unterminated string literal (detected at line 4) (2338372753.py, line 4)

In [42]:
def check_twitter_urls(urls):
    for url in urls:
        if url.startswith("https://twitter.com/"):
            return 1
    return 0

In [43]:
n =0
for urls in df['urls']:
    n = n+check_twitter_urls(urls)



In [46]:
print(n/461)

0.13232104121475055


In [49]:
df_meet = df[df["Predicted Label"]==df["True Label"]]

In [50]:
df_diff = df[df["Predicted Label"] != df['True Label']]

In [52]:
print(f"Text: {df_diff['Text'][18]}, Keywords: {df_diff['Keywords'][18]} Topics: {df_diff['Topics'][18]}")

Text: Quote of the Day, April 15, 2024;

Three more quotes chosen by Jose Ramos:

"We’ve lived so long under the spell of hierarchy—from god-kings to feudal lords to party bosses—that only recently have we awakened to see not only that “regular” citizens have the capacity for self-governance, but that without their engagement our huge global crises cannot be addressed. The changes needed for human society simply to survive, let alone thrive, are so profound that the only way we will move toward them is if we ourselves, regular citizens, feel meaningful ownership of solutions through direct engagement. Our problems are too big, interrelated, and pervasive to yield to directives from on high."

—Frances Moore Lappé, Time for Progressives to Grow Up 

Someone who has been immersed in orthodoxy needs to experience a figureground reversal in order to gain perspective. This can’t come from encountering just a few heterodox thoughts, but only from a new encompassing architecture of interconne

In [62]:
item_types_whitelist = [
    "bookSection",
    "journalArticle",
    "preprint",
    "book",
    "manuscript",
    "thesis",
    "presentation",
    "conferencePaper",
    "report",
]


topics_whitelist = [
    "technology",
    "science",
    "academia",
    "research",
    "design",
    "climate",
    "sustainability",
    "software & hardware",
    "philosophy",
    "health",
    "culture",
    "economics",
    "business",
    "finance",
    "literature",
]


def apply_research_filter_keywords_down(row):
    # if any item types on the whitelist, pass automatically
    if len(set(row['Ref item types']).intersection(set(item_types_whitelist))) > 0:
        return 'research'

    # if item types inconclusive, use scoring system
    score = 0

    # if research related topics not present = 1 point
    if len(set(row['Topics']).intersection(set(topics_whitelist))) == 0:
        score += 2


    # if no references present = 1 point
    if row['ref_count'] == 0:
        score += 1

    if score >= 2:
        return 'not_research'
    else:
        return 'reaserch'

In [63]:
df1 = df
df1['Predicted Label'] = df1.apply(lambda row: apply_research_filter_keywords_down(row),axis=1)

In [67]:
results[0].research_keyword

'academic'